# Problem 2
Authors: Fabian Boemer, Sid Murching, Suraj Nair, Alex Cui

In [85]:
import numpy as np
import matplotlib.pyplot as plt
import random

## 2C:
Fill in these functions to train your SVD

In [149]:
def grad_U(Ui, Yij, Vj, reg, eta):
    """
    Takes as input Ui (the ith row of U), a training point Yij, the column
    vector Vj (jth column of V^T), reg (the regularization parameter lambda),
    and eta (the learning rate).

    Returns the gradient of the regularized loss function with
    respect to Ui multiplied by eta.
    """
    t1 = reg * Ui
    
    Vj = np.squeeze(np.asarray(Vj))
    Ui = np.squeeze(np.asarray(Ui))
    
    UT = np.dot(Vj,Ui)
    t2 = Vj * (Yij - UT)
    return eta*(t1 - t2)   

def grad_V(Ui, Yij, Vj, reg, eta):
    """
    Takes as input the column vector Vj (jth column of V^T), a training point Yij,
    Ui (the ith row of U), reg (the regularization parameter lambda),
    and eta (the learning rate).

    Returns the gradient of the regularized loss function with
    respect to Vj multiplied by eta.
    """
    t1 = reg * Vj
    
    Vj = np.squeeze(np.asarray(Vj))
    Ui = np.squeeze(np.asarray(Ui))
    
    UT = np.dot(Vj,Ui)
    t2 = Ui * (Yij - UT)
    return eta*(t1 - t2)

def get_err(U, V, Y, reg=0.0):
    """
    Takes as input a matrix Y of triples (i, j, Y_ij) where i is the index of a user,
    j is the index of a movie, and Y_ij is user i's rating of movie j and
    user/movie matrices U and V.

    Returns the mean regularized squared-error of predictions made by
    estimating Y_{ij} as the dot product of the ith row of U and the jth column of V^T.
    """
    totErr = 0
    U_transpose = np.matrix(np.transpose(U))
    rest = np.asarray(U_transpose * np.matrix(V))
    #assert(rest.shape == Y.shape)
    
    for row in range(len(Y)):
        for col in range(len(Y[0])):
            if Y[row][col] > 0.1:
                totErr += pow(Y[row][col] - rest[row][col],2)
    return sqrt(totErr)

def train_model(M, N, K, eta, reg, Y, eps=0.0001, max_epochs=300):
    """
    Given a training data matrix Y containing rows (i, j, Y_ij)
    where Y_ij is user i's rating on movie j, learns an
    M x K matrix U and N x K matrix V such that rating Y_ij is approximated
    by (UV^T)_ij.

    Uses a learning rate of <eta> and regularization of <reg>. Stops after
    <max_epochs> epochs, or once the magnitude of the decrease in regularized
    MSE between epochs is smaller than a fraction <eps> of the decrease in
    MSE after the first epoch.

    Returns a tuple (U, V, err) consisting of U, V, and the unregularized MSE
    of the model.
    """
    
    print("M: ", M)
    print("K: ", K)
    
    i = []
    j = []
    Y_ij = []
    allLoss = []

    for x in range(len(Y)):
        i1, j1, y_ij1 = Y[x]
        i.append(i1)
        j.append(j1)
        Y_ij.append(y_ij1)
        #print(i1, "/", j1, "/", y_ij1)
        
    U = np.random.uniform(-.5,.5, size=(M, K))
    V = np.random.uniform(-.5,.5, size=(N, K))
        
    for _ in range(max_epochs):
        rp = random.randint(0, len(i))
        rU = int(i[rp]-1)
        rV = int(j[rp]-1)
        
        U[rU] += grad_U( U[rU], Y_ij[rp], V[rV], reg, eta)
        V[rV] += grad_V( U[rU], Y_ij[rp], V[rV], reg, eta)
        allLoss.append(get_err(U, V, Y))
       
    return (U, V, allLoss)

## 2D:
Run the cell below to get your graphs

In [ ]:
Y_train = np.loadtxt('./data/train.txt').astype(int)
Y_test = np.loadtxt('./data/test.txt').astype(int)

M = max(max(Y_train[:,0]), max(Y_test[:,0])).astype(int) # users
N = max(max(Y_train[:,1]), max(Y_test[:,1])).astype(int) # movies
print("Factorizing with ", M, " users, ", N, " movies.")
Ks = [10,20,30,50,100]

reg = 0.0
eta = 0.03 # learning rate
E_in = []
E_out = []

# Use to compute Ein and Eout
for K in Ks:
    U,V, err = train_model(M, N, K, eta, reg, Y_train)
    E_in.append(err)
    E_out.append(get_err(U, V, Y_test))

plt.plot(Ks, E_in, label='$E_{in}$')
plt.plot(Ks, E_out, label='$E_{out}$')
plt.title('Error vs. K')
plt.xlabel('K')
plt.ylabel('Error')
plt.legend()
plt.savefig('2d.png')

## 2E:
Run the cell below to get your graphs. This might take a long time to run, but it should take less than 2 hours. I would encourage you to validate your 2C is correct.

In [ ]:
Y_train = np.loadtxt('./data/train.txt').astype(int)
Y_test = np.loadtxt('./data/test.txt').astype(int)

M = max(max(Y_train[:,0]), max(Y_test[:,0])).astype(int) # users
N = max(max(Y_train[:,1]), max(Y_test[:,1])).astype(int) # movies
Ks = [10,20,30,50,100]

regs = [10**-4, 10**-3, 10**-2, 10**-1, 1]
eta = 0.03 # learning rate
E_ins = []
E_outs = []

# Use to compute Ein and Eout
for reg in regs:
    E_ins_for_lambda = []
    E_outs_for_lambda = []

    for k in Ks:
        print("Training model with M = %s, N = %s, k = %s, eta = %s, reg = %s"%(M, N, k, eta, reg))
        U,V, e_in = train_model(M, N, k, eta, reg, Y_train)
        E_ins_for_lambda.append(e_in)
        eout = get_err(U, V, Y_test)
        E_outs_for_lambda.append(eout)

    E_ins.append(E_ins_for_lambda)
    E_outs.append(E_outs_for_lambda)


# Plot values of E_in across k for each value of lambda
for i in range(len(regs)):
    plt.plot(Ks, E_ins[i], label='$E_{in}, \lambda=$'+str(regs[i]))
plt.title('$E_{in}$ vs. K')
plt.xlabel('K')
plt.ylabel('Error')
plt.legend()
plt.savefig('2e_ein.png')	
plt.clf()

# Plot values of E_out across k for each value of lambda
for i in range(len(regs)):
    plt.plot(Ks, E_outs[i], label='$E_{out}, \lambda=$'+str(regs[i]))
plt.title('$E_{out}$ vs. K')
plt.xlabel('K')
plt.ylabel('Error')
plt.legend()	
plt.savefig('2e_eout.png')		
